# $T_1$ Experiment

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see [our tutorials](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/index.html) if you need to create your setup and qubits for the first time. 

You can run this notebook on real hardware in the lab. However, if you don't have the hardware at your disposal, you can also run the notebook "as is" using an emulated session (see below). 

If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background

In this how-to guide, you'll perform a measurement to determine the relaxation time, $T_1$, for a given qubit using the `T1` experiment included in the LabOne Q Applications Library. In a $T_1$ experiment, you'll vary the delay time, $\tau$, between an initial excitation pulse and the subsequent measurement of the qubit's state. This allows you to observe how the qubit relaxes from the excited state, $\ket{1}$, back to the ground state, $\ket{0}$, over time.

For a qubit transitioning from the excited state, $\ket{1}$, to the ground state, $\ket{0}$, the population decay follows an exponential behavior according to:

$$\rho(\ket{1}) = e^{-\tau/T_1}$$



### Imports

You'll start by importing `laboneq.simple`.

In [ ]:
from laboneq.simple import *

### Define your experimental setup

Let's define our experimental setup. We will need:

* a [DeviceSetup](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/concepts/00_set_up_equipment.html#devicesetup)

* `n` [TunableTransmonQubits](https://docs.zhinst.com/labone_q_user_manual/applications_library/reference/qpu_types/tunable_transmon.html#laboneq_applications.qpu_types.tunable_transmon.TunableTransmonQubit)

* a set of [TunableTransmonOperations](https://docs.zhinst.com/labone_q_user_manual/applications_library/reference/qpu_types/tunable_transmon.html#laboneq_applications.qpu_types.tunable_transmon.TunableTransmonOperations)

* a [QPU](https://docs.zhinst.com/labone_q_user_manual/core/reference/dsl/quantum.html#laboneq.dsl.quantum.qpu.QPU)

Here, we will be brief. We will mainly provide the code to obtain these objects. To learn more, check out these other tutorials:

* Details about [defining your experimental setup](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/getting_started.html)

* [Qubit parameters and how quantum operations use them](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/quantum_operations.html)

* [Quantum operations in general](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/03_sections_pulses_and_quantum_operations/concepts/08_quantum_operations.html)

* [Logbooks and data saving with Workflows](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/logbooks.html)

We will use 3 `TunableTransmonQubits` in this guide. Change this number to the one describing your setup.

In [ ]:
number_of_qubits = 3

#### DeviceSetup

This guide requires a setup that can drive and readout tunable transmon qubits. Your setup could contain an SHFQC+ instrument, or an SHFSG and an SHFQA instruments. Here, we will use an SHFQC+ with 6 signal generation channels and a PQSC.

If you have used LabOne Q before and already have a `DeviceSetup` for your setup, you can reuse that.

If you do not have a `DeviceSetup`, you can create one using the code below. Just change the device numbers to the ones in your rack and adjust any other input parameters as needed.

In [ ]:
# Setting get_zsync=True below, automatically detects the zsync ports of the PQCS that
# are used by the other instruments in this descriptor.
# Here, we are not connected to instruments, so we set this flag to False.
from laboneq.contrib.example_helpers.generate_descriptor import generate_descriptor

descriptor = generate_descriptor(
    pqsc=["DEV10001"],
    shfqc_6=["DEV12001"],
    number_data_qubits=number_of_qubits,
    multiplex=True,
    number_multiplex=number_of_qubits,
    include_cr_lines=False,
    get_zsync=False,  # set to True when at a real setup
    ip_address="localhost",
)
setup = DeviceSetup.from_descriptor(descriptor, "localhost")

#### Qubits

We will generate 3 `TunableTransmonQubits` from the logical signal groups in our `DeviceSetup`. The names of the logical signal groups, `q0`, `q1`, `q2`, will be the UIDs of the qubits. Moreover, the qubits will have the same logical signal lines as the ones of the logical signal groups in the `DeviceSetup`.

In [ ]:
from laboneq_applications.qpu_types.tunable_transmon import (
    TunableTransmonQubit,
)

qubits = TunableTransmonQubit.from_device_setup(setup)

In [ ]:
for q in qubits:
    print("-------------")
    print("Qubit UID:", q.uid)
    print("Qubit logical signals:")
    for sig, lsg in q.signals.items():
        print(f"  {sig:<10} ('{lsg:>10}')")

Configure the qubit parameters to reflect the properties of the qubits on your QPU using the following code:

In [ ]:
for q in qubits:
    q.parameters.ge_drive_pulse["sigma"] = 0.25
    q.parameters.readout_amplitude = 0.5
    q.parameters.reset_delay_length = 1e-6
    q.parameters.readout_range_out = -25
    q.parameters.readout_lo_frequency = 7.4e9

qubits[0].parameters.drive_lo_frequency = 6.4e9
qubits[0].parameters.resonance_frequency_ge = 6.3e9
qubits[0].parameters.resonance_frequency_ef = 6.0e9
qubits[0].parameters.readout_resonator_frequency = 7.0e9

qubits[1].parameters.drive_lo_frequency = 6.4e9
qubits[1].parameters.resonance_frequency_ge = 6.5e9
qubits[1].parameters.resonance_frequency_ef = 6.3e9
qubits[1].parameters.readout_resonator_frequency = 7.3e9

qubits[2].parameters.drive_lo_frequency = 6.0e9
qubits[2].parameters.resonance_frequency_ge = 5.8e9
qubits[2].parameters.resonance_frequency_ef = 5.6e9
qubits[2].parameters.readout_resonator_frequency = 7.2e9

#### Quantum Operations

Create the set of `TunableTransmonOperations`:

In [ ]:
from laboneq_applications.qpu_types.tunable_transmon import TunableTransmonOperations

qops = TunableTransmonOperations()

#### QPU

Create the `QPU` object from the qubits and the quantum operations

In [ ]:
from laboneq.dsl.quantum import QPU

qpu = QPU(qubits=qubits, quantum_operations=qops)

#### Alternatively, load from a file

If you you already have a `DeviceSetup` and a `QPU` stored in `.json` files, you can simply load them back using the code below:

```python
from laboneq import serializers

setup = serializers.load(full_path_to_device_setup_file)
qpu = serializers.load(full_path_to_qpu_file)

qubits = qpu.qubits
qops = qpu.quantum_operations
```

### Connect to Session

In [ ]:
session = Session(setup)
session.connect(do_emulation=True)  # do_emulation=False when at a real setup

### Create a `FolderStore` for Saving Data

The experiment `Workflows` can automatically save the inputs and outputs of all their tasks to the folder path we specify when instantiating the `FolderStore`. Here, we choose the current working directory.

In [ ]:
# import FolderStore from the `workflow` namespace of LabOne Q, which was imported
# from `laboneq.simple`
from pathlib import Path

folder_store = workflow.logbook.FolderStore(Path.cwd())

We disable saving in this guide. To enable it, simply run `folder_store.activate()`.

In [ ]:
folder_store.deactivate()

### Optional: Configure the LoggingStore

You can also activate/deactivate the `LoggingStore`, which is used for displaying the `Workflow` logging information in the notebook; see again the [tutorial on Recording Experiment Workflow Results](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/logbooks.html) for details.

Displaying the `Workflow` logging information is activated by default, but here we deactivate it to shorten the outputs, which are not very meaningful in emulation mode.

**We recommend that you do not deactivate the Workflow logging in practice.**

In [ ]:
from laboneq.workflow.logbook import LoggingStore

logging_store = LoggingStore()
logging_store.deactivate()

### Running the Experiment Workflow

You'll now instantiate the experiment workflow and run it. For more details on what experiment workflows are and what tasks they execute, see the [Experiment Workflows tutorial](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/experiment_workflows.html).

You'll start by importing `numpy`, the $T_1$ (lifetime_measurement) experiment workflow from `laboneq_applications`, as well as `plot_simulation` for inspecting the experiment sequence.

In [ ]:
import numpy as np
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

from laboneq_applications.experiments import lifetime_measurement

Let's first create the options class for the $T_1$ experiment and inspect it using the `show_fields` function from the `workflow` namespace of LabOne Q, which was imported from `laboneq.simple`:

In [ ]:
options = lifetime_measurement.experiment_workflow.options()
workflow.show_fields(options)

Notice that, unless we change it:

- the experiment is run in `AcquisitionType.INTEGRATION` and `AveragingMode.CYCLIC`, using 1024 averages (`count`)
- the experiment is run on the qubit $|g\rangle \leftrightarrow |e\rangle$ (`transition`)
- calibration traces are added (`use_cal_traces=True`) for the states $|g\rangle$ and $|e\rangle$ (`cal_states`) 
- the analysis workflow will run automatically (`do_analysis=True`)
- the figures produced by the analysis are automatically closed (`close_figures=True`)
- the qubit parameters will not be updated (`update=False`)

Here, let's disable closing the figures produced by the analysis so we see them in the cell output. Note however that the fit attempted by the analysis routine in emulation mode will not be representative, because we do not acquire data from a real experiment.

In [ ]:
options.close_figures(False)

Now we run the experiment workflow on the first two qubits in parallel.

In [ ]:
# our qubits live here in the demo setup:
qubits = qpu.qubits

exp_workflow = lifetime_measurement.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=[qubits[0], qubits[1]],
    delays=[np.linspace(0, 50e-6, 50), np.linspace(0, 50e-6, 50)],
    options=options
)

workflow_results = exp_workflow.run()

#### Inspect the Tasks That Were Run

In [ ]:
for t in workflow_results.tasks:
    print(t)

#### Inspect the Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
compiled_experiment = workflow_results.tasks["compile_experiment"].output
plot_simulation(compiled_experiment, length=50e-6)

#### Inspecting the Source Code of the Pulse-Sequence Creation Task

You can inspect the source code of the `create_experiment` task defined in `lifetime_measurement` to see how the experiment pulse sequence is created using quantum operations. To learn more about the latter, see the [Quantum Operations tutorial](../../../../tutorials/sources/quantum_operations.ipynb).

In [ ]:
lifetime_measurement.create_experiment.src

To learn more about how to work with experiment `Workflows`, check out the [Experiment Workflows tutorial](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/experiment_workflows.html).

Here, let's briefly inspect the analysis-workflow results.

#### Analysis Results

Let's check what tasks were run as part of the analysis workflow:

In [ ]:
analysis_workflow_results = workflow_results.tasks["analysis_workflow"]
for t in analysis_workflow_results.tasks:
    print(t)

We can access the qubit parameters extracted by the analysis from the output of the analysis-workflow:

In [ ]:
from pprint import pprint

pprint(analysis_workflow_results.output)  # noqa: T203

Check out the [Experiment Workflows tutorial](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/experiment_workflows.html) to see how you can manually update the qubit parameters to these new values, or reset them to the old ones. 

Great! You've now run your $T_1$ experiment. Check out other experiments in this manual to keep characterizing your qubits.